# Ciencia de datos en la nube: la forma "Azure ML SDK"

## 1. Introducción

Los científicos de datos y los desarrolladores de inteligencia artificial utilizan el SDK de Azure Machine Learning para crear y ejecutar flujos de trabajo de aprendizaje automático con el servicio Azure Machine Learning. Puede interactuar con el servicio en cualquier entorno de Python, incluidos Jupyter Notebooks, Visual Studio Code o su IDE de Python favorito.

Las áreas clave del SDK incluyen:

* Explore, prepare y administre el ciclo de vida de sus conjuntos de datos utilizados en experimentos de aprendizaje automático.
* Administre recursos en la nube para monitorear, registrar y organizar sus experimentos de aprendizaje automático.
* Entrene modelos localmente o utilizando recursos de la nube, incluido el entrenamiento de modelos acelerado por GPU.
* Utilice aprendizaje automático automatizado, que acepta parámetros de configuración y datos de entrenamiento. Itera automáticamente a través de algoritmos y configuraciones de hiperparámetros para encontrar el mejor modelo para ejecutar predicciones.
* Implemente servicios web para convertir sus modelos entrenados en servicios RESTful que se puedan consumir en cualquier aplicación.

### Importaciones específicas de Azure Machine Learning SDK

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

### Inicializar espacio de trabajo
Inicialice un objeto de área de trabajo a partir de una configuración persistente. Asegúrese de que el archivo de configuración esté presente en .\config.json

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Creación de un experimento de Azure ML
Vamos a crear un experimento denominado 'aml-experiment' en el espacio de trabajo que acabamos de inicializar.

In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

### Creación de un clúster de proceso
Tendrás que crear un destino de proceso para la ejecución de AutoML.

In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

### Datos
Asegúrese de que ha cargado el conjunto de datos en Azure ML y de que la clave tiene el mismo nombre que el conjunto de datos.

In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

### Configuración de AutoML

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

### Ejecución de AutoML

In [ ]:
remote_run = experiment.submit(automl_config)
     

RunDetails(remote_run).show()

### Guarde el mejor modelo

In [ ]:
best_run, fitted_model = remote_run.get_output()
     

best_run.get_properties()
     

model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

### Implemente el mejor modelo
Ejecute el código siguiente para implementar el mejor modelo. Puede ver el estado de la implementación en Azure ML Portal. Este paso puede tardar unos minutos.

In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

### Consumir el punto de conexión
Puede agregar entradas al siguiente ejemplo de entrada.

In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))
     

response = aci_service.run(input_data=test_sample)
response
     